In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

%matplotlib inline
plt.style.use('ggplot')

### First task: Convert raw data into dataframes:
1. Sales, etc. by inventory_id
2. Aggregate sales, etc. by data

In [2]:
path = '../data/lemon_haze_18.csv'
test_path = '../data/test_shard.csv'

In [ ]:
df_raw = pd.read_csv(path, parse_dates=[2, 3])
df_raw.head()

In [ ]:
df_raw.info()

In [ ]:
by_inv_id = df_raw.copy()

In [ ]:
by_inv_id['days_inventory'] = by_inv_id['latest_rtl_sale'] - by_inv_id['wholesale_date']

In [ ]:
by_inv_id['gross_profit'] = by_inv_id['ttl_retail_sales'] - by_inv_id['wholesale_cogs']

In [ ]:
by_inv_id.info()

In [ ]:
q = by_inv_id.loc[0, 'days_inventory']

In [ ]:
by_inv_id['avg_daily_sales'] = \
by_inv_id['ttl_retail_sales'] / by_inv_id['days_inventory'].apply(lambda x: float(x.days))

In [ ]:
by_inv_id['avg_daily_gross'] = \
by_inv_id['gross_profit'] / by_inv_id['days_inventory'].apply(lambda x: float(x.days))

In [ ]:
by_inv_id.head()

In [ ]:
cols = [
    'wa_inventory_id',
    'generic_strain_id',
    'wholesale_date',
    'latest_rtl_sale',
    'days_inventory',
    'wholesale_cogs',
    'ttl_retail_sales',
    'gross_profit',
    'avg_daily_sales',
    'avg_daily_gross',
    'units_sold'
]

In [ ]:
by_inv_id = by_inv_id[cols]

In [ ]:
by_inv_id.head()

In [ ]:
def convert_to_by_inventory_id_sales(df):
    """Convert raw strain df to output sales by inventory_id df
    INPUT RAW DF:
        Data columns (total 7 columns):
        wa_inventory_id      2565 non-null int64
        generic_strain_id    2565 non-null int64
        wholesale_date       2563 non-null datetime64[ns]
        latest_rtl_sale      2514 non-null datetime64[ns]
        wholesale_cogs       2565 non-null float64
        ttl_retail_sales     2514 non-null float64
        units_sold           2514 non-null float64
    """
    

In [ ]:
shard = by_inv_id[:5][['wa_inventory_id', 'wholesale_date', 'latest_rtl_sale', 'avg_daily_sales', 'avg_daily_gross']]

In [ ]:
shard

In [ ]:
# This is our index
q_range = pd.date_range(shard['wholesale_date'].min(), shard['latest_rtl_sale'].max())

In [ ]:
q_piv = pd.DataFrame(0, index=q_range, columns=shard['wa_inventory_id'])

In [ ]:
q_piv.head()
# From here, it's a boolean: index date in the date range embedded in column inventory_id?

In [ ]:
# Making the first factor: a boolean dataframe indexed by each date in overall range
# boolean being whether the date index is found in the date_range when the inventory_id
# was in retail inventory

matrix_list = []
for date in q_range.strftime('%Y-%m-%d'):
    date_bool = []
    for record in shard.iterrows():
        date_bool.append(date in pd.date_range(record[1][1], record[1][2]))
    matrix_list.append(date_bool)


shard_bool = pd.DataFrame(matrix_list, index=q_range, columns=shard['wa_inventory_id'])

In [ ]:
# Making the second factor: the vector of daily average sales per inventory_id
sales = shard[['avg_daily_sales', 'avg_daily_gross']].values.reshape(len(shard['avg_daily_sales']), 2)

In [ ]:
shard_bool.shape, sales.shape

In [ ]:
ttl_sales_n_profit = shard_bool.dot(sales) # ugly, but done. Could add the gross profit column

In [ ]:
ttl_sales_n_profit.columns = ['ttl_sales', 'ttl_gross']

In [ ]:
ttl_sales_n_profit.head(15)

In [ ]:
import os
import sys


In [101]:
from convert_to_time_series import TimeSeriesData

In [113]:
lemon_haze = TimeSeriesData(test_path)

In [114]:
%time lemon_haze.construct()

CPU times: user 24.5 ms, sys: 24.9 ms, total: 49.5 ms
Wall time: 91.6 ms


In [115]:
lemon_haze._bool_matrix

,0,1,2,3,4,5,6,7,8,9,...,162,163,164,165,166,167,168,169,170,171
0,True,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
1,True,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2,True,True,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
3,True,True,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
4,True,True,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
5,True,True,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
6,True,True,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
7,True,True,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
8,True,True,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
9,True,True,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


In [45]:
ts = lemon_haze.time_series
raw = lemon_haze.raw_df
inv = lemon_haze.by_inv_id

In [46]:
ts.describe()

,ttl_sales,ttl_gross_profit
count,504.000000,504.000000
mean,567.260738,377.614660
std,1164.743274,805.932737
min,1.770833,1.008929
25%,22.082567,12.398938
50%,354.418997,219.468618
75%,495.694243,315.480473
max,9964.563845,6744.314410


In [12]:
raw.head()

,wa_inventory_id,generic_strain_id,wholesale_date,latest_rtl_sale,wholesale_cogs,ttl_retail_sales,units_sold
0,6033240110000277,18,2014-07-17,2014-09-19,2700.0,7740.50,150
1,6033240110000284,18,2014-07-19,2014-08-07,1200.0,4436.00,100
2,6033240110000316,18,2014-07-30,2014-08-03,1760.0,5501.87,100
3,6033240110000314,18,2014-07-30,2014-08-08,1760.0,5313.60,96
4,6033240110000315,18,2014-07-30,2014-08-04,1760.0,5535.00,100


In [13]:
inv.head()

,wa_inventory_id,generic_strain_id,wholesale_date,latest_rtl_sale,days_inventory,wholesale_cogs,ttl_retail_sales,gross_profit,avg_daily_sales,avg_daily_gross,units_sold
0,6033240110000277,18,2014-07-17,2014-09-19,64 days,2700.0,7740.50,5040.50,120.945312,78.757812,150
1,6033240110000284,18,2014-07-19,2014-08-07,19 days,1200.0,4436.00,3236.00,233.473684,170.315789,100
2,6033240110000316,18,2014-07-30,2014-08-03,4 days,1760.0,5501.87,3741.87,1375.467500,935.467500,100
3,6033240110000314,18,2014-07-30,2014-08-08,9 days,1760.0,5313.60,3553.60,590.400000,394.844444,96
4,6033240110000315,18,2014-07-30,2014-08-04,5 days,1760.0,5535.00,3775.00,1107.000000,755.000000,100


In [88]:
rows = np.array([3,3,3,3,3])
lowers = np.array([1,4,3])
uppers = np.array([4,3,5])

In [80]:
rs, low, up = np.meshgrid(rows, lowers, uppers)

In [81]:
low

array([[[1, 1, 1],
        [1, 1, 1],
        [1, 1, 1],
        [1, 1, 1],
        [1, 1, 1]],

       [[4, 4, 4],
        [4, 4, 4],
        [4, 4, 4],
        [4, 4, 4],
        [4, 4, 4]],

       [[3, 3, 3],
        [3, 3, 3],
        [3, 3, 3],
        [3, 3, 3],
        [3, 3, 3]]])

In [91]:
np.tile(rows, len(lowers)).reshape

array([3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3])

In [68]:
rs2, low2 = np.meshgrid(rows, lowers)

In [73]:
low2

array([[1, 1, 1, 1, 1],
       [4, 4, 4, 4, 4],
       [3, 3, 3, 3, 3]])

In [36]:
thing_one = y >= z
thing_two = y <= z

In [37]:
thing_one, thing_two

(array([[False,  True,  True],
        [False,  True,  True],
        [False,  True,  True],
        [False,  True,  True],
        [False,  True,  True]], dtype=bool), array([[ True, False,  True],
        [ True, False,  True],
        [ True, False,  True],
        [ True, False,  True],
        [ True, False,  True]], dtype=bool))

In [38]:
final = thing_one * thing_two

In [39]:
final

array([[False, False,  True],
       [False, False,  True],
       [False, False,  True],
       [False, False,  True],
       [False, False,  True]], dtype=bool)

In [51]:
inv[inv['avg_daily_sales'] > 500][['wa_inventory_id', 'avg_daily_sales', 'avg_daily_gross']].\
sort_values(['avg_daily_gross'], ascending=False)

,wa_inventory_id,avg_daily_sales,avg_daily_gross
5,6033240110000330,1786.716667,1200.050000
2,6033240110000316,1375.467500,935.467500
4,6033240110000315,1107.000000,755.000000
20,6033240110000456,894.086667,701.780000
6,6033240110000313,922.500000,629.166667
9,6033240110000309,730.492500,499.242500
7,6033240110000308,719.542500,488.292500
12,6033240110000310,719.536250,488.286250
8,6033240110000311,710.896250,479.646250
3,6033240110000314,590.400000,394.844444


In [52]:
dates = np.array([1,2,3,4,5])

In [92]:
%time 2**5

CPU times: user 5 µs, sys: 8 µs, total: 13 µs
Wall time: 37 µs


32